<a href="https://colab.research.google.com/github/KaifAhmad1/code-test/blob/main/Deepfake_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### **Deepfake and Manipulated Media Analysis Data Download**

In [1]:
!pip install -q datasets pandas pillow tqdm huggingface_hub
!apt-get install -y ffmpeg  # Install FFmpeg for video processing
!pip install -q decord  # Install decord after FFmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.


In [2]:
import os
import pandas as pd
from pathlib import Path
from datasets import load_dataset
from tqdm import tqdm
from PIL import Image
from huggingface_hub import login
import getpass

In [3]:
# Authenticate Hugging Face
HF_TOKEN = getpass.getpass("Enter your Hugging Face token: ")
login(HF_TOKEN)

Enter your Hugging Face token: ··········


In [4]:
# Set download directory
BASE_DIR = Path.home() / "Downloads" / "deepfake_dataset"
BASE_DIR.mkdir(parents=True, exist_ok=True)
metadata = []

In [5]:
def download_data(dataset_name, category_name, file_ext, key):
    """Downloads and saves data (images, videos, audio)"""
    print(f"\n📥 Downloading {category_name} Data...")
    try:
        dataset = load_dataset(dataset_name, split="train")
    except Exception as e:
        print(f"❌ Error loading {dataset_name}: {e}")
        return

    save_dir = BASE_DIR / category_name
    save_dir.mkdir(parents=True, exist_ok=True)

    for category, label in [("real", 0), ("fake", 1)]:
        (save_dir / category).mkdir(parents=True, exist_ok=True)
        samples = dataset.filter(lambda x: x["label"] == label).select(range(20))

        for i, sample in enumerate(tqdm(samples, desc=f"Saving {category} {category_name}")):
            file_path = save_dir / category / f"{category}_{i:03d}.{file_ext}"
            # Handle both in-memory bytes and file paths
            if isinstance(sample[key], dict) and 'bytes' in sample[key]:
                content = sample[key]['bytes']
            else:
                with open(sample[key], 'rb') as f:
                    content = f.read()
            with open(file_path, "wb") as f:
                f.write(content)

            metadata.append({
                "modality": category_name,
                "category": category,
                "filename": file_path.name,
                "file_path": str(file_path),
                "source_dataset": dataset_name
            })

def save_metadata():
    """Saves metadata as CSV"""
    pd.DataFrame(metadata).to_csv(BASE_DIR / "metadata.csv", index=False)
    print("✅ Metadata saved")

def main():
    # Download datasets
    download_data("thenewsupercell/fixed-fakeavceleb", "audio", "wav", "audio")
    download_data("mkhLlamaLearn/dfdc", "videos", "mp4", "video")
    download_data("taohu/faceforensics_h5", "images", "jpg", "image")
    save_metadata()
    print(f"\n🎉 Dataset Collection Complete! Data stored in {BASE_DIR}")

In [6]:
if __name__ == "__main__":
    main()


📥 Downloading audio Data...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Filter:   0%|          | 0/15999 [00:00<?, ? examples/s]

RuntimeError: Error reading /kaggle/working/new_video_dataset/train/FakeVideo-FakeAudio/00001_0_id06152_wavtolip.mp4...